In [95]:
import numpy as np
import math
from tqdm import tqdm, trange

In [96]:
def to_bin(x, d):
    assert d >= 0
    
    if d == 0:
        return ""
    else:
        return f'{x:0{d}b}'

In [98]:
def T(n):
    return int((n+1) * n / 2)

In [106]:
n = 5
num_pairs = T(n-1)

In [99]:
def conflicts(G_str, selected_str):
    d = len(selected_str)
    
    if d == 0:
        return False

    to_check = G_str[T(d-1):T(d-1)+d]
    for x, y in zip(selected_str, to_check):
        if x == "1" and y == "1":
            return True

In [100]:
def find_edge(j):
    assert j >= 0  # j is one-indexed
    
    temp = 0
    i = 1
    while temp + i < j + 1:
        temp += i
        i += 1
    return j - temp, i

In [101]:
comp_ratios = {}

for p in tqdm(np.linspace(0, 1, num=50)):
    first_iter = True
    for G_int in range(2**num_pairs):
        G_str = to_bin(G_int, num_pairs)

        # optimal offline algorithm
        # TODO optimization: go in order of decreasing selected set size and break once you find a valid one
        opt_val = 0
        for i in range(2**n):
            selected_str = to_bin(i, n)
            valid_sel = True
            for j, e in enumerate(G_str):
                if e == "1":
                    x, y = find_edge(j)
                    if selected_str[x] == "1" and selected_str[y] == "1":
                        valid_sel = False
                        break
            if valid_sel:
                val = 0
                for node in selected_str:
                    if node == "1":
                        val += 1
                opt_val = max(opt_val, val)


        # probabilistic greedy algorithm
        F = {}

        for i in range(2**n):
            F[to_bin(i, n)] = 0

        for d in range(n-1,-1,-1):
            for i in range(2**d):
                selected_str = to_bin(i, d)

                if not conflicts(G_str, selected_str):  # all selected nodes are not joined to current node
                    res = p * (1 + F[selected_str + "1"]) + (1-p) * F[selected_str + "0"]
                else:
                    res = F[selected_str + "0"]

                F[selected_str] = res

        greedy_val = F[""]
        
        
        ratio = opt_val / greedy_val if greedy_val > 0 else math.inf
        
        if first_iter:
            best_G = G_str
            comp_ratio = ratio
            first_iter = False
        elif ratio > comp_ratio:
            best_G = G_str
            comp_ratio = ratio

    comp_ratios[p] = (best_G, comp_ratio)

100%|███████████████████████████████████████████████████████████████| 50/50 [00:46<00:00,  1.08it/s]


In [102]:
p = min(comp_ratios, key=lambda string: comp_ratios.get(string)[1])

In [103]:
G_str, comp_ratio = comp_ratios[p]

In [104]:
print(f"with {p=:.4g}, graph {G_str} gives competitive ratio {comp_ratio:.5g}")

with p=0.551, graph 1111101100 gives competitive ratio 2.651


In [105]:
for j, e in enumerate(G_str):
    if e == "1":
        x, y = find_edge(j)
        print((x+1, y+1))

(1, 2)
(1, 3)
(2, 3)
(1, 4)
(2, 4)
(1, 5)
(2, 5)
